In [291]:
from sage.all import *
p = 0xD3915

F = GF(p)

In [292]:

import random

def get_elliptic_parameters(size):
    # random r
    while True:
        random_bit = random.getrandbits(size)
        binary_rand = '0' + bin(random_bit)[2:].zfill(size)
        r = int(binary_rand, 2)

        # random B same manner as r
        random_bit = random.getrandbits(size)
        binary_rand = '0' + bin(random_bit)[2:].zfill(size)
        b = int(binary_rand, 2)

        R.<a> = PolynomialRing(F)
        L = (r*b**2-a**3).roots()

        for u in L:
            check = (4*u[0]**2 + 27*b**2) % p
            if check != 0:
                a = u[0]
                return a, b, r
    

a, b ,r = get_elliptic_parameters(20)
a, b, r

(823816, 805546, 35274)

### CURVE DEFINITION

In [314]:
## CURVE DEFINITION
p = 0xD3915
F = GF(p)
a = 738492
b = 694682
r = 926251
ECurv = EllipticCurve(F, [a, b])

print(f"Curve: {ECurv}")

Curve: Elliptic Curve defined by y^2 = x^3 + 738492*x + 694682 over Finite Field of size 866581


In [294]:
# Test case 1
p1 = ECurv(184224, 74658)
p2 = ECurv(428817, 567437)

p1+p2
p1

(184224 : 74658 : 1)

In [295]:
# Test case 2
p1 = ECurv(24069, 233375)
p2 = ECurv(249867, 503874)

p1+p2

(847840 : 636963 : 1)

In [296]:
# # Test case 3
p1 = ECurv(40300, 763164)
p2 = ECurv(18900, 353015)

p1+p2

# ECurv.random_point()

(548652 : 419566 : 1)

### POINT MULTIPLY TESTS

In [297]:
p1 = ECurv(264320, 549393)

4*p1

(38956 : 83726 : 1)

### POINT ADD IMPL

In [298]:

from sage.all import *

def ell_add(E, P1, P2):
    a, b, p = E
    if P1 == "inf":
        return P2
    if P2 == "inf":
        return P1
    x1, y1 = P1
    x2, y2 = P2

    if x1 == x2 and y1 == (p - y2):
        return "inf"

    if P1 == P2:
        if y1 == 0:
            return "inf"
        lam = ((3 * (x1 ** 2) + a) * inverse_mod(2 * y1, p)) % p
    else:
        lam = ((y2 - y1) * inverse_mod(x2 - x1, p)) % p

    x3 = (lam ** 2 - x1 - x2) % p
    y3 = (-lam * x3 - y1 + lam * x1) % p
    return (x3, y3)

### RHO POLLARD

In [299]:

from typing import Tuple


class Group_parameters:
    def __init__(self, E, P, Q) -> None:
        self.E: Tuple[int, int, int] = E # a, b, p of Elliptic Curve
        self.P = P # generator
        self.Q = Q # Pub Key


class Triple:
    def __init__(self, X, a, b) -> None:
        self.X = X  # Point at Elliptic Curve
        self.a = a  # just a number
        self.b = b  # just a number

    def __str__(self) -> str:
        return f"x = {self.X}, a = {self.a}, b = {self.b}"


def f(triple: Triple, group: Group_parameters) -> Triple:

    x_of_xpoint = int(triple.X[0])

    p = group.P.order()

    if x_of_xpoint % 3 == 0:
        X = triple.X + group.Q
        a = triple.a
        b = (triple.b + 1)
        # check 
        if (X != a * group.P + b * group.Q):
            print(f"3 xab: {X} {a} {b}")
        return Triple(X, a, b)
    if x_of_xpoint % 3 == 1:
        X = 2 * triple.X 
        a = (triple.a * 2)
        b = (triple.b * 2)
        if (X != a * group.P + b * group.Q):
            print(f"3 xab: {X} {a} {b}")
        return Triple(X, a, b)
    else:
        X = triple.X + group.P
        a = (triple.a + 1)
        b = triple.b
        if (X != a * group.P + b * group.Q):
            print(f"3 xab: {X} {a} {b}")
        return Triple(X, a, b)


def main(g: Group_parameters, t1):
    i = 1

    t2 = f(t1, g)

    print("%s %s | %s %s" % (i, t1, 2 * i, t2))

    i = 2
    while (t1.X != t2.X) or (((t2.b - t1.b) % g.P.order()) == 0):
        t1 = f(t1, g)
        t2 = f(f(t2, g), g)
        print("%s %s | %s %s" % (i, t1, 2 * i, t2))
        i = i + 1

    print(f"Found:\nt1: {t1}\nt2: {t2}")
    print(f"Iteration nr: {i}")

    x = ((t1.a - t2.a) * inverse_mod((t2.b - t1.b), g.P.order())) % g.P.order()

    print(x)

In [318]:

# TEST DATA

ECurv = EllipticCurve(GF(p), [2,2])
print(ECurv)
a = 2
b = 2


import random
P = ECurv.random_point()
print(P)

print(P.order())

Q = 21271 * P

print(Q)

E = (int(a), int(b), int(p))
g = Group_parameters(E, P, Q)

a_i = random.randint(2, P.order()-2)
b_i = random.randint(2, P.order()-2)
X_i = a_i * P + b_i * Q
print(f"X_0: {X_i}")
print(f"a_0: {a_i}")
print(f"b_0: {b_i}")
t = Triple(X_i, a_i, b_i)
x = main(g, t)

Elliptic Curve defined by y^2 = x^3 + 2*x + 2 over Finite Field of size 866581
(13614 : 653781 : 1)
78821
(560419 : 740155 : 1)
X_0: (267093 : 825347 : 1)
a_0: 5953
b_0: 26139
1 x = (267093 : 825347 : 1), a = 5953, b = 26139 | 2 x = (635529 : 29967 : 1), a = 5953, b = 26140
2 x = (635529 : 29967 : 1), a = 5953, b = 26140 | 4 x = (195554 : 289432 : 1), a = 5953, b = 26142
3 x = (15396 : 556913 : 1), a = 5953, b = 26141 | 6 x = (249498 : 77069 : 1), a = 5954, b = 26143
4 x = (195554 : 289432 : 1), a = 5953, b = 26142 | 8 x = (765788 : 326020 : 1), a = 5954, b = 26145
5 x = (275622 : 602927 : 1), a = 5954, b = 26142 | 10 x = (462080 : 366938 : 1), a = 11910, b = 52290
6 x = (249498 : 77069 : 1), a = 5954, b = 26143 | 12 x = (540524 : 80604 : 1), a = 23822, b = 104580
7 x = (645348 : 673691 : 1), a = 5954, b = 26144 | 14 x = (353021 : 466567 : 1), a = 47646, b = 209160
8 x = (765788 : 326020 : 1), a = 5954, b = 26145 | 16 x = (188313 : 861425 : 1), a = 95294, b = 418320
9 x = (99415 : 4096